In [36]:
import itertools
import random

In [53]:
class City:
    def __init__(self, name, connections):
        self.name = name
        self.connections = connections
        # array of tuples containing City objs and integers representing costs
        
    def __str__(self):
        return self.name
        
    def getDistance(self, other):
        for tuple in self.connections:
            if tuple[0] == other: return tuple[1]
        
        return -1

    def printConnections(self):
        print("----- Connections of {} -----".format(self.name))
        for tuple in self.connections:
            print("({}, {})".format(tuple[0].name, tuple[1]))

def connectCities(c1, c2, distance):
    '''Function to connect two unique cities, returns True/False on Succss/Failure'''
    if(c1 == c2): return False
    for tuples in c1.connections:
        if (tuples[0] == c2): return False
    for tuples in c2.connections:
        if (tuples[0] == c1): return False
    c1.connections.append((c2, distance))
    c2.connections.append((c1, distance))
    
    return True

def calcDist(perm):
    '''Function to calculate total distance in provided permutation
        Returns -1 if there is no path, otherwise returns total distance'''
    totalDistance = 0
    for i in range(1, len(perm)):
        dist = perm[i-1].getDistance(perm[i])
        if (dist != -1):
            totalDistance += dist
        else:
            return -1
    
    return totalDistance

In [54]:
one = City("1", [])
two = City("2", [])
three = City("3", [])
four = City("4", [])
five = City("5", [])
six = City("6", [])

In [55]:
connectCities(one, two, 3)
connectCities(one, three, 7)
connectCities(one, four, 4)
connectCities(one, five, 2)
connectCities(one, six, 6)
connectCities(two, three, 4)
connectCities(two, four, 4)
connectCities(two, five, 6)
connectCities(two, six, 5)
connectCities(three, four, 4)
connectCities(three, five, 7)
connectCities(three, six, 4)
connectCities(four, five, 4)
connectCities(four, six, 2)
connectCities(five, six, 3)

True

In [97]:
# hill climbing solution (4.3a)
originalperm = [one, two, three, four, five, six, one]
middle = [two, three, four, five, six]
random.shuffle(middle)
originalperm = [one, middle[0], middle[1], middle[2], middle[3], middle[4], one]
print("-- Randomized Initial State --")
print(*originalperm)
print(calcDist(originalperm))

def findBestPerm(perm):
    temp = perm.copy()
    
    best = temp
    bestDist = calcDist(temp)
    # find natural next step by switching other elements in the list w/ the 2nd element in the list
    for i in range(2, len(temp)-1):
        temp = perm.copy()
        temp[1], temp[i] = temp[i], temp[1]
        
        # print("- - - - -")
        # for city in temp:
        #    print(city.name)
        
        tempDist = calcDist(temp)
        if tempDist < bestDist:
            best = temp
            bestDist = tempDist
    
    return best.copy()

smallestDist = calcDist(originalperm)
currPerm = originalperm.copy()
while(True):
    
    currPerm = findBestPerm(currPerm)
    currDist = calcDist(currPerm)
    
    # print("-- new iteration --")
    # print("best perm in iteration")
    # for city in currPerm:
    #     print(city.name)
    # print("Dist: {}".format(currDist))
    
    if(currDist < smallestDist):
        smallestDist = currDist
    else:
        break

print("-- Found Either Local or Global Maximum --")
print(*currPerm)
print("Dist: {}".format(smallestDist))

-- Randomized Initial State --
1 6 3 4 2 5 1
26
-- Found Either Local or Global Maximum --
1 2 3 4 6 5 1
Dist: 18


In [83]:
# functions for genetic algorithm solution (4.3b)
originalperm = [one, two, three, four, five, six, one]

# rank population in order of shortest -> longest total distance
def rankPop(population):
    population.sort(key=calcDist)
    
def selection(numTourn, numMutate, population):
    toReturn = []
    rankPop(population)
    for i in range(numTourn):
        toReturn.append(population[i])
    for i in range(numMutate):
        rand = random.choice(toReturn).copy()
        #randomly switch two values
        i1, i2 = random.sample(range(1, len(rand)-1), 2)
        rand[i1], rand[i2] = rand[i2], rand[i1]
        toReturn.append(rand)
    rankPop(toReturn)
    return toReturn

def breed(parent1, parent2):
    #randomly generate gene segment to take from parent 1
    index1 = random.randint(1, (len(parent1)-1))
    index2 = random.randint(1, (len(parent1)-1))
    
    start = min(index1, index2)
    end = max(index1, index2)
    
    p1genes = []
    for i in range(start, end):
        p1genes.append(parent1[i])
        
    p2genes = [item for item in parent2 if item not in p1genes]
    
    child = []
    
    p1count = 0;
    p2count = 0;
    
    for i in range(len(parent1)):
        if (i >= start) and (i < end):
            child.append(p1genes[p1count])
            p1count += 1
        else:
            child.append(p2genes[p2count])
            p2count += 1
        
    
    return child

def breedPopulation(matingPool, numKept):
    children = []
    length = len(matingPool) - numKept
    
    pool = matingPool.copy()
    rankPop(pool)
    
    for i in range(numKept):
        children.append(pool[i])
    
    random.shuffle(matingPool)
    for i in range(length):
        child = breed(matingPool[i], matingPool[-1])
        children.append(child)
        
    rankPop(children)
    return children

def nextGen(currGen, elite, numTourn, numMutate):
    pop = currGen.copy()
    rankPop(pop)
    selected = selection(numTourn, numMutate, pop)
    children = breedPopulation(selected, elite)
    rankPop(children)
    return children


In [94]:
# output for genetic algorithm (4.3b)
print("-- Creating Randomized Initial Population --")
populationSize = 10
population = []
middle = [two, three, four, five, six]

print("Population Size: {}".format(populationSize))

# create init pop
for i in range(populationSize):
    random.shuffle(middle)
    temp = middle.copy()
    temp.insert(0, one)
    temp.append(one)
    population.append(temp)

rankPop(population)

numTourn = 8
numMutate = 2
elite = 4
generations = 10
print("-- Generational Settings --")
print("Tournament Selection Number:\t\t{}".format(numTourn))
print("Number of Mutations per Generation:\t{}".format(numMutate))
print("Number of Pops Kept with Mutation:\t{}".format(elite))
print("Number of Generations:\t\t\t{}".format(generations))
lowestDist = calcDist(population[0])
lowestIndv = population[0]
for i in range(generations):
    population = nextGen(population, elite, numTourn, numMutate)
    dist = calcDist(population[0])
    if dist < lowestDist:
        lowestDist = dist
        lowestIndv = population[0]
    print(*population[0])
    print(dist)
    print("---")
        
print("\nOptimal Path:")
print(*lowestIndv)
print("Total Distance: {}".format(lowestDist))

-- Creating Randomized Initial Population --
Population Size: 10
-- Generational Settings --
Tournament Selection Number:		8
Number of Mutations per Generation:	2
Number of Pops Kept with Mutation:	4
Number of Generations:			10
1 3 2 6 4 5 1
24
---
1 3 2 6 4 5 1
24
---
1 2 3 6 4 5 1
19
---
1 2 3 6 4 5 1
19
---
1 2 3 6 4 5 1
19
---
1 2 3 6 4 5 1
19
---
1 2 3 6 4 5 1
19
---
1 2 3 4 6 5 1
18
---
1 2 3 4 6 5 1
18
---
1 2 3 4 6 5 1
18
---

Optimal Path:
1 2 3 4 6 5 1
Total Distance: 18
